In [ ]:
#采用CPU进行训练
import torch.nn as nn
import torch
import torch.optim as optim
import numpy as np
import struct
import matplotlib.pyplot as plt


def data_fetch_preprocessing():
    train_image = open('train-images.idx3-ubyte', 'rb')
    test_image = open('t10k-images.idx3-ubyte', 'rb')
    train_label = open('train-labels.idx1-ubyte', 'rb')
    test_label = open('t10k-labels.idx1-ubyte', 'rb')

    magic, n = struct.unpack('>II',
                             train_label.read(8))
    # 原始数据的标签
    y_train_label = np.array(np.fromfile(train_label,
                                         dtype=np.uint8), ndmin=1)
    y_train = np.ones((10, 60000)) * 0.01
    for i in range(60000):
        y_train[y_train_label[i]][i] = 0.99

    # 测试数据的标签
    magic_t, n_t = struct.unpack('>II',
                                 test_label.read(8))
    y_test = np.fromfile(test_label,
                         dtype=np.uint8).reshape(10000, 1)
    # print(y_train[0])
    # 训练数据共有60000个
    # print(len(labels))
    magic, num, rows, cols = struct.unpack('>IIII', train_image.read(16))
    x_train = np.fromfile(train_image, dtype=np.uint8).reshape(len(y_train_label), 784)

    magic_2, num_2, rows_2, cols_2 = struct.unpack('>IIII', test_image.read(16))
    x_test = np.fromfile(test_image, dtype=np.uint8).reshape(len(y_test), 784)
    # print(x_train.shape)
    # 可以通过这个函数观察图像
    # data=x_train[:,0].reshape(28,28)
    # plt.imshow(data,cmap='Greys',interpolation=None)
    # plt.show()

    # 关闭打开的文件
    train_image.close()
    train_label.close()
    test_image.close()
    test_label.close()

    return x_train, y_train_label, x_test, y_test


class convolution_neural_network(nn.Module):
    def __init__(self):
        super(convolution_neural_network, self).__init__()

        # 定义卷积层
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0),  # 28x28x1-->24x24x6
            nn.Sigmoid(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 12x12x6
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),  # 8x8x16
            nn.Sigmoid(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 4x4x16
        )
        self.fc = nn.Sequential(
            nn.Linear(in_features=256, out_features=120),
            nn.Sigmoid(),
            nn.Linear(in_features=120, out_features=84),
            nn.Sigmoid(),
            nn.Linear(in_features=84, out_features=10),
        )

    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output


if __name__ == '__main__':
    # 获取数据
    x_train, y_train, x_test, y_test = data_fetch_preprocessing()
    x_train = x_train.reshape(60000, 1, 28, 28)
    # 建立模型实例
    LeNet = convolution_neural_network() 
    ##加入判断是CPU训练还是GPU训练
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    LeNet.to(device)

    # plt.imshow(x_train[2][0], cmap='Greys', interpolation=None)
    # plt.show()
    # 交叉熵损失函数
    loss_function = nn.CrossEntropyLoss()
    loss_list = []
    optimizer = optim.Adam(params=LeNet.parameters(), lr=0.001)
    # epoch = 5
    for e in range(3):
        precision = 0
        for i in range(60000):
            prediction = LeNet(torch.tensor(x_train[i]).float().reshape(-1, 1, 28, 28))
            # print(prediction)
            # print(torch.from_numpy(y_train[i]).reshape(1,-1))
            # exit(-1)rk
            if torch.argmax(prediction) == y_train[i]:
                precision += 1
            loss = loss_function(prediction, torch.tensor([y_train[i]]).long())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_list.append(loss)
        print('第%d轮迭代，loss=%.3f，准确率：%.3f' % (e, loss_list[-1],precision/60000))
        ##使用eva1()将模型设置为“推理模式”



In [2]:
#采用GPU进行训练
import torch.nn as nn
import torch
import torch.optim as optim
import numpy as np
import struct
import matplotlib.pyplot as plt


def data_fetch_preprocessing():
    train_image = open('train-images.idx3-ubyte', 'rb')
    test_image = open('t10k-images.idx3-ubyte', 'rb')
    train_label = open('train-labels.idx1-ubyte', 'rb')
    test_label = open('t10k-labels.idx1-ubyte', 'rb')

    magic, n = struct.unpack('>II',
                             train_label.read(8))
    # 原始数据的标签
    y_train_label = np.array(np.fromfile(train_label,
                                         dtype=np.uint8), ndmin=1)
    y_train = np.ones((10, 60000)) * 0.01
    for i in range(60000):
        y_train[y_train_label[i]][i] = 0.99

    # 测试数据的标签
    magic_t, n_t = struct.unpack('>II',
                                 test_label.read(8))
    y_test = np.fromfile(test_label,
                         dtype=np.uint8).reshape(10000, 1)
    # print(y_train[0])
    # 训练数据共有60000个
    # print(len(labels))
    magic, num, rows, cols = struct.unpack('>IIII', train_image.read(16))
    x_train = np.fromfile(train_image, dtype=np.uint8).reshape(len(y_train_label), 784)

    magic_2, num_2, rows_2, cols_2 = struct.unpack('>IIII', test_image.read(16))
    x_test = np.fromfile(test_image, dtype=np.uint8).reshape(len(y_test), 784)
    # print(x_train.shape)
    # 可以通过这个函数观察图像
    # data=x_train[:,0].reshape(28,28)
    # plt.imshow(data,cmap='Greys',interpolation=None)
    # plt.show()

    # 关闭打开的文件
    train_image.close()
    train_label.close()
    test_image.close()
    test_label.close()

    return x_train, y_train_label, x_test, y_test


class convolution_neural_network(nn.Module):
    def __init__(self):
        super(convolution_neural_network, self).__init__()

        # 定义卷积层
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0),  # 28x28x1-->24x24x6
            nn.Sigmoid(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 12x12x6
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),  # 8x8x16
            nn.Sigmoid(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 4x4x16
        )
        self.fc = nn.Sequential(
            nn.Linear(in_features=256, out_features=120),
            nn.Sigmoid(),
            nn.Linear(in_features=120, out_features=84),
            nn.Sigmoid(),
            nn.Linear(in_features=84, out_features=10),
        )

    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output


if __name__ == '__main__':
    # 获取数据
    x_train, y_train, x_test, y_test = data_fetch_preprocessing()
    x_train = x_train.reshape(60000, 1, 28, 28)
    
    # 建立模型实例
    model = convolution_neural_network()
    
    # 加入判断是CPU训练还是GPU训练
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # 超参数
    batch_size = 256
    learning_rate = 0.001
    num_epochs = 10
    
    # 交叉熵损失函数
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)
    
    # 数据集转换为Tensor并移到GPU
    x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
    
    # 迭代训练
    for epoch in range(num_epochs):
        total_loss = 0.0
        correct_predictions = 0
    
        for i in range(0, len(x_train), batch_size):
            x_batch = x_train_tensor[i:i+batch_size]
            y_batch = y_train_tensor[i:i+batch_size]
            #print(x_batch)
    
            prediction = model(x_batch)
            loss = loss_function(prediction, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
            total_loss += loss.item()
            correct_predictions += (torch.argmax(prediction, dim=1) == y_batch).sum().item()
    
        average_loss = total_loss / (len(x_train) / batch_size)
        accuracy = correct_predictions / len(x_train)
    
        print(f'Epoch {epoch+1}, Average Loss: {average_loss:.3f}, Accuracy: {accuracy:.3f}')
    # 模型保存
    #torch.save(LeNet, 'model.pt')
    # 模型加载
    #model = torch.load('model.pt')

    # 保存模型参数
    #torch.save(model.state_dict(), "model.pt")
    ##使用eva1()将模型设置为“推理模式”
    # set the model to inference mode 
    model.eval() 

    # 在导出之前将模型移回 CPU
    model.to("cpu")
    
    # Let's create a dummy input tensor
    dummy_input = torch.randn(1, 1, 28, 28, requires_grad=True)
    
    # Export the model
    torch.onnx.export(model, dummy_input, "mnist_model.onnx",
                      export_params=True, opset_version=12,
                      do_constant_folding=True,
                      input_names=['input'],
                      output_names=['output'],
                     )
    
    print(" ")
    print('Model has been converted to ONNX')


    # # 导出之前的设备设置等代码...
    
    # # Let's create a dummy input tensor in HWC format
    # dummy_input_hwc = torch.randn(1, 28, 28, 1, requires_grad=True)
    
    # # Transpose the dummy input from HWC to CHW format
    # dummy_input_chw = dummy_input_hwc.permute(0, 3, 1, 2)
    
    # # Export the model
    # torch.onnx.export(model, dummy_input_chw, "Networkhwc.onnx",
    #                   export_params=True, opset_version=12,
    #                   do_constant_folding=True,
    #                   input_names=['input'],
    #                   output_names=['output'])

    
    print(" ")
    print('Model has been converted to ONNX')


Epoch 1, Average Loss: 1.545, Accuracy: 0.508
Epoch 2, Average Loss: 0.396, Accuracy: 0.882
Epoch 3, Average Loss: 0.231, Accuracy: 0.934
Epoch 4, Average Loss: 0.164, Accuracy: 0.952
Epoch 5, Average Loss: 0.131, Accuracy: 0.961
Epoch 6, Average Loss: 0.112, Accuracy: 0.967
Epoch 7, Average Loss: 0.097, Accuracy: 0.971
Epoch 8, Average Loss: 0.086, Accuracy: 0.975
Epoch 9, Average Loss: 0.078, Accuracy: 0.976
Epoch 10, Average Loss: 0.072, Accuracy: 0.978
============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

 
Model has been converted to ONNX
 
Model has been converted to ONNX


In [ ]:
#onnx导出
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
#import trainModel as tm

##获取输入参数
#data = iter(tm.test_dataloader)
#dummy_inputs, labels = next(data)  #直接通过数据集来起作用

dummy_inputs = torch.rand(1, 1, 28, 28).float()
print(dummy_inputs.shape)
 
##加载模型
model = convolution_neural_network()
model.load_state_dict(torch.load("model.pt"))

#print(model)
 
 
##加载的模型测试效果
outputs = model(dummy_inputs)
print(outputs)
##预测返回的是两列，第一列是下标就是0-9的值，第二列为预测值，下面的dim=1就是找维度1（第二列）最大值输出
# _, predicted = torch.max(outputs.data, dim=1)
# print(_)
# print(predicted)
# outlabels = predicted.numpy().tolist()
# print(outlabels)


##定义输出输出的参数名
input_name = ["input"]
output_name = ["output"] 
onnx_name = 'mnist_model.onnx'

torch.onnx.export(
    model,
    dummy_inputs,
    onnx_name,
    verbose=True,
    input_names=input_name,
    output_names=output_name,
    opset_version=12  # 设置 Opset 版本为 11
)


In [ ]:
import onnxruntime as ort
import numpy as np
import torch

# 加载 ONNX 模型
onnx_path = 'model.onnx'
sess = ort.InferenceSession(onnx_path, providers=['CUDAExecutionProvider'])  # 'CPUExecutionProvider'

# 创建一个随机输入数据，然后将其转换为 PyTorch 的 Tensor
random_input = torch.rand(1, 1, 28, 28).float()
#input_data = random_input.cpu().numpy()  # 将 Tensor 转换为 NumPy 数组

example_image = cv2.imread('mnist_image.jpg', cv2.IMREAD_GRAYSCALE)

example_image = np.expand_dims(example_image, axis=0)  # Add channel dimension
img=example_image.astype(np.float32)
img = np.expand_dims(img, 0).astype(np.float32)

print(img.shape)

# 获取模型的输入名称
input_name = sess.get_inputs()[0].name

# 进行推理
outputs = sess.run(None, {input_name: img})
print(outputs)
# 提取模型的输出张量，并使用 torch.argmax()
output_tensor = torch.tensor(outputs[0])  # 假设第一个输出是你想要获取的
argmax_result = torch.argmax(output_tensor)

print(argmax_result)

In [ ]:
import random
#以下使用opencv进行推理
import cv2
import numpy as np
import struct
import matplotlib.pyplot as plt

# 加载数据集
def data_fetch_preprocessing():
    train_image = open('train-images.idx3-ubyte', 'rb')
    test_image = open('t10k-images.idx3-ubyte', 'rb')
    train_label = open('train-labels.idx1-ubyte', 'rb')
    test_label = open('t10k-labels.idx1-ubyte', 'rb')

    magic, n = struct.unpack('>II',
                             train_label.read(8))
    # 原始数据的标签
    y_train_label = np.array(np.fromfile(train_label,
                                         dtype=np.uint8), ndmin=1)
    y_train = np.ones((10, 60000)) * 0.01
    for i in range(60000):
        y_train[y_train_label[i]][i] = 0.99

    # 测试数据的标签
    magic_t, n_t = struct.unpack('>II',
                                 test_label.read(8))
    y_test = np.fromfile(test_label,
                         dtype=np.uint8).reshape(10000, 1)
    # print(y_train[0])
    # 训练数据共有60000个
    # print(len(labels))
    magic, num, rows, cols = struct.unpack('>IIII', train_image.read(16))
    x_train = np.fromfile(train_image, dtype=np.uint8).reshape(len(y_train_label), 784)

    magic_2, num_2, rows_2, cols_2 = struct.unpack('>IIII', test_image.read(16))
    x_test = np.fromfile(test_image, dtype=np.uint8).reshape(len(y_test), 784)
    # print(x_train.shape)
    # 可以通过这个函数观察图像
    # data=x_train[:,0].reshape(28,28)
    # plt.imshow(data,cmap='Greys',interpolation=None)
    # plt.show()

    # 关闭打开的文件
    train_image.close()
    train_label.close()
    test_image.close()
    test_label.close()

    return x_train, y_train_label, x_test, y_test
# 要抽取的图像数量
num_samples = 1

# 获取数据集长度
dataset_length = len(x_test)

# 生成随机抽样的索引
random_indices = random.sample(range(dataset_length), num_samples)

# Load the neural network model
model_path = 'model.onnx'
net = cv2.dnn.readNet(model_path)

# 循环抽取的图像进行推理
for i in random_indices:
    input_data = x_test[i].astype(np.float32).reshape(28, 28)  # 转换为 float 类型

    # 将输入数据转换为 Blob
    blob = cv2.dnn.blobFromImage(input_data, scalefactor=1.0, size=(28, 28), mean=0.5, swapRB=False)

    # 设置输入数据
    net.setInput(blob)

    # 进行前向传播
    outputs = net.forward()

    # 提取模型的输出张量，并使用 np.argmax()
    output_tensor = outputs[0]  # 假设第一个输出是你想要获取的
    argmax_result = np.argmax(output_tensor)

    # 打印输入图像、标签和输出结果
    input_image = x_test[i].reshape(28, 28)
    true_label = y_test[i][0]
    predicted_label = argmax_result

    plt.imshow(input_image, cmap='Greys', interpolation=None)
    plt.title(f'True Label: {true_label}, Predicted Label: {predicted_label}')
    plt.show()

In [ ]:
#加入计时模块
import random
import timeit
import numpy as np

# 要抽取的图像数量
num_samples = 1

# 获取数据集长度
dataset_length = len(x_test)

# 生成随机抽样的索引
random_indices = random.sample(range(dataset_length), num_samples)

# 定义前处理函数
def preprocess(input_data):
    blob = cv2.dnn.blobFromImage(input_data, scalefactor=1.0, size=(28, 28), mean=0.5, swapRB=False)
    return blob

# 定义推理函数
def inference(net, blob):
    net.setInput(blob)
    outputs = net.forward()
    return outputs

# 初始化时间列表
preprocessing_times = []
inference_times = []
total_times = []
# Load the neural network model
model_path = 'model.onnx'
net = cv2.dnn.readNet(model_path)
# 循环抽取的图像进行推理
for i in random_indices:
    input_data = x_test[i].astype(np.float32).reshape(28, 28)  # 转换为 float 类型

    # 计时前处理时间（毫秒）
    preprocessing_time_ms = timeit.timeit(lambda: preprocess(input_data), number=1) * 1000
    preprocessing_times.append(preprocessing_time_ms)

    # 计时推理时间（毫秒）
    inference_time_ms = timeit.timeit(lambda: inference(net, preprocess(input_data)), number=1) * 1000
    inference_times.append(inference_time_ms)

    # 计算总时间（毫秒）
    total_time_ms = preprocessing_time_ms + inference_time_ms
    total_times.append(total_time_ms)

    # 提取模型的输出张量，并使用 np.argmax()
    output_tensor = inference(net, preprocess(input_data))[0]  # 假设第一个输出是你想要获取的
    argmax_result = np.argmax(output_tensor)

    # 打印计时信息
    print(f"Image {i + 1} - Preprocessing Time: {preprocessing_time_ms:.6f} ms, Inference Time: {inference_time_ms:.6f} ms")

    # 打印输入图像、标签和输出结果
    input_image = x_test[i].reshape(28, 28)
    true_label = y_test[i][0]
    predicted_label = argmax_result
        # 保存图像


    plt.imshow(input_image, cmap='Greys', interpolation=None)
    plt.title(f'True Label: {true_label}, Predicted Label: {predicted_label}')
    plt.show()
    output_image_path = 'mnist_image.jpg'  # 替换为您的输出路径
    #plt.savefig(output_image_path, bbox_inches='tight', pad_inches=0)
    
    # 调整颜色通道顺序，然后保存图像
    output_image_path = 'mnist_image.jpg'  # 替换为您的输出路径

# 将颜色通道顺序从 BGR 调整为 RGB
    input_image_rgb = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)

# 保存图像
    cv2.imwrite(output_image_path, input_image_rgb)  
    
# 计算平均时间
average_preprocessing_time = np.mean(preprocessing_times)
average_inference_time = np.mean(inference_times)
average_total_time = np.mean(total_times)

print(f"Average Preprocessing Time: {average_preprocessing_time:.6f} ms")
print(f"Average Inference Time: {average_inference_time:.6f} ms")
print(f"Average Total Time: {average_total_time:.6f} ms")


In [ ]:
#加入计时模块
import random
import timeit
import numpy as np

# 要抽取的图像数量
num_samples = 2

# 获取数据集长度
dataset_length = len(x_test)

# 生成随机抽样的索引
random_indices = random.sample(range(dataset_length), num_samples)

# 定义前处理函数
def preprocess(input_data):
    blob = cv2.dnn.blobFromImage(input_data, scalefactor=1.0, size=(28, 28), mean=0.5, swapRB=False)
    return blob

# 定义推理函数
def inference(net, blob):
    net.setInput(blob)
    outputs = net.forward()
    return outputs

# 初始化时间列表
preprocessing_times = []
inference_times = []
total_times = []
# Load the neural network model
model_path = 'model.onnx'
net = cv2.dnn.readNet(model_path)
# 循环抽取的图像进行推理
for i in random_indices:
    input_data = x_test[i].astype(np.float32).reshape(28, 28)  # 转换为 float 类型

    # 计时前处理时间（毫秒）
    preprocessing_time_ms = timeit.timeit(lambda: preprocess(input_data), number=1) * 1000
    preprocessing_times.append(preprocessing_time_ms)

    # 计时推理时间（毫秒）
    inference_time_ms = timeit.timeit(lambda: inference(net, preprocess(input_data)), number=1) * 1000
    inference_times.append(inference_time_ms)

    # 计算总时间（毫秒）
    total_time_ms = preprocessing_time_ms + inference_time_ms
    total_times.append(total_time_ms)

    # 提取模型的输出张量，并使用 np.argmax()
    output_tensor = inference(net, preprocess(input_data))[0]  # 假设第一个输出是你想要获取的
    argmax_result = np.argmax(output_tensor)

    # 打印计时信息
    print(f"Image {i + 1} - Preprocessing Time: {preprocessing_time_ms:.6f} ms, Inference Time: {inference_time_ms:.6f} ms")

    # 打印输入图像、标签和输出结果
    input_image = x_test[i].reshape(28, 28)
    true_label = y_test[i][0]
    predicted_label = argmax_result

    plt.imshow(input_image, cmap='Greys', interpolation=None)
    plt.title(f'True Label: {true_label}, Predicted Label: {predicted_label}')
    plt.show()

# 计算平均时间
average_preprocessing_time = np.mean(preprocessing_times)
average_inference_time = np.mean(inference_times)
average_total_time = np.mean(total_times)

print(f"Average Preprocessing Time: {average_preprocessing_time:.6f} ms")
print(f"Average Inference Time: {average_inference_time:.6f} ms")
print(f"Average Total Time: {average_total_time:.6f} ms")

In [11]:
#采用GPU进行训练
import torch.nn as nn
import torch
import torch.optim as optim
import numpy as np
import struct
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import MNIST

class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

        # Calculate mean and std of the dataset
        mean = torch.zeros(1)  # Only one channel for grayscale images
        std = torch.zeros(1)
        for sample, _ in self.data:
            sample = self.transform(sample) if self.transform else sample
            mean += sample.mean()
            std += sample.std()
        mean /= len(self.data)
        std /= len(self.data)

        # Define a transformation to normalize data using calculated mean and std
        self.normalize = transforms.Normalize(mean=mean.item(), std=std.item())
        print(mean.item(), std.item())

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample, label = self.data[idx]
        if self.transform:
            sample = self.transform(sample)
        sample = self.normalize(sample)  # Apply normalization
        return sample, label
class convolution_neural_network(nn.Module):
    def __init__(self):
        super(convolution_neural_network, self).__init__()

        # 定义卷积层
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0),  # 28x28x1-->24x24x6
            nn.Sigmoid(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 12x12x6
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),  # 8x8x16
            nn.Sigmoid(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 4x4x16
        )
        self.fc = nn.Sequential(
            nn.Linear(in_features=256, out_features=120),
            nn.Sigmoid(),
            nn.Linear(in_features=120, out_features=84),
            nn.Sigmoid(),
            nn.Linear(in_features=84, out_features=10),
        )

    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output

def train(model, train_loader, loss_function, optimizer, device):
    model.train()
    
    total_loss = 0.0
    correct_predictions = 0
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        prediction = model(data)
        loss = loss_function(prediction, target)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        correct_predictions += (torch.argmax(prediction, dim=1) == target).sum().item()

    average_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / len(train_loader.dataset)
    return average_loss, accuracy

if __name__ == '__main__':
    # 超参数
    learning_rate = 0.01
    num_epochs = 50
    batch_size = 2048

    # Load MNIST dataset
    train_dataset = MNIST(root='./mnist/', train=True, download=True,
                          transform=None)  # Don't apply ToTensor transformation here   
    # Create a custom dataset
    train_dataset_custom = CustomDataset(train_dataset, transform=transforms.ToTensor())
    
    # Create a DataLoader
    train_loader = DataLoader(train_dataset_custom, batch_size=batch_size, shuffle=True, num_workers=10, pin_memory=True)

    # 建立模型实例
    model = convolution_neural_network()

    # 加入判断是CPU训练还是GPU训练
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)
    model=model.to(device)

    # 交叉熵损失函数
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)

    # 迭代训练
    for epoch in range(num_epochs):
        average_loss, accuracy = train(model, train_loader, loss_function, optimizer, device)
        print(f'Epoch {epoch+1}, Average Loss: {average_loss:.3f}, Accuracy: {accuracy:.3f}')

    # 在导出之前将模型移回 CPU
    model.to("cpu")
    
    # Let's create a dummy input tensor
    dummy_input = torch.randn(1, 1, 28, 28, requires_grad=True)
    
    # Export the model
    torch.onnx.export(model, dummy_input, "mnist_model.onnx",
                      export_params=True, opset_version=12,
                      do_constant_folding=True,
                      input_names=['input'],
                      output_names=['output'],
                     )
    
    print(" ")
    print('Model has been converted to ONNX')
   

0.13065974414348602 0.3015038073062897
cuda:0
Epoch 1, Average Loss: 2.266, Accuracy: 0.128
Epoch 2, Average Loss: 1.713, Accuracy: 0.361
Epoch 3, Average Loss: 0.905, Accuracy: 0.686
Epoch 4, Average Loss: 0.367, Accuracy: 0.887
Epoch 5, Average Loss: 0.189, Accuracy: 0.943
Epoch 6, Average Loss: 0.123, Accuracy: 0.963
Epoch 7, Average Loss: 0.091, Accuracy: 0.972
Epoch 8, Average Loss: 0.075, Accuracy: 0.978
Epoch 9, Average Loss: 0.064, Accuracy: 0.981
Epoch 10, Average Loss: 0.054, Accuracy: 0.984
Epoch 11, Average Loss: 0.051, Accuracy: 0.985
Epoch 12, Average Loss: 0.045, Accuracy: 0.987
Epoch 13, Average Loss: 0.040, Accuracy: 0.988
Epoch 14, Average Loss: 0.035, Accuracy: 0.990
Epoch 15, Average Loss: 0.031, Accuracy: 0.991
Epoch 16, Average Loss: 0.028, Accuracy: 0.992
Epoch 17, Average Loss: 0.026, Accuracy: 0.993
Epoch 18, Average Loss: 0.022, Accuracy: 0.994
Epoch 19, Average Loss: 0.022, Accuracy: 0.993
Epoch 20, Average Loss: 0.018, Accuracy: 0.995
Epoch 21, Average Loss: